### Imports

In [1]:
import collections
import gc
import json
from matplotlib import pyplot as plt
import numpy as np
import random
import tools
reload(tools)
# gc.collect()

<module 'tools' from 'tools.pyc'>

### Process January to May Train data

We first load the original graph data for Jan-May, with all the users, but without the weights.

In [2]:
subredditIdToName = tools.read_subreddit_names("../bigData/subredditIdToName")
infilenamePrefix = "../bigData/userSubreddit/RC_2018-0"
userIdToOldSubreddits = collections.defaultdict(set)
for i in range(1, 6):
    infilename = infilenamePrefix + str(i)
    tools.getUserIdToSubreddits(infilename, userIdToOldSubreddits, subredditIdToName)

Processing ../bigData/userSubreddit/RC_2018-01
Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 5000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000
Processed 13000000
Processed 14000000
Processed 15000000
Processed 16000000
Processed 17000000
Processing ../bigData/userSubreddit/RC_2018-02
Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 5000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000
Processed 13000000
Processed 14000000
Processed 15000000
Processed 16000000
Processed 17000000
Processing ../bigData/userSubreddit/RC_2018-03
Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 5000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000
Processed 13

Next, we save a list of users with less than 3 subreddits in the months of Jan to May, and a list of subreddits with at least 10 users in the months of Jan to May.

In [3]:
with open('../bigData/finalGeneration/userIdsToFilter', 'w') as outfile:
    for userId, subreddits in userIdToOldSubreddits.iteritems():
        if len(subreddits) < 3:
            outfile.write("{}\n".format(userId))

In [5]:
subredditUserCounts = collections.defaultdict(int)
for userId, subreddits in userIdToOldSubreddits.iteritems():
    for subreddit in subreddits:
        subredditUserCounts[subreddit] += 1
with open('../bigData/finalGeneration/subredditIdsToKeep', 'w') as outfile:
    for subreddit, count in subredditUserCounts.iteritems():
        if count >= 10:
            outfile.write("{}\n".format(subreddit))

Now reload the graph with the weights and filters and save it.

In [5]:
userIdsToFilter = set()
tools.readUserIdFilters("../bigData/finalGeneration/userIdsToFilter", userIdsToFilter)
subredditIdsToKeep = set()
tools.readUserIdFilters("../bigData/finalGeneration/subredditIdsToKeep", subredditIdsToKeep)

infilenamePrefix = "../bigData/userSubreddit/RC_2018-0"
userIdToOldSubreddits = collections.defaultdict(lambda: collections.defaultdict(int))
for i in range(1, 6):
    infilename = infilenamePrefix + str(i)
    tools.getUserIdToSubreddits(infilename, userIdToOldSubreddits, subredditIdsToKeep, userIdsToFilter, True)
print "Total users: {}".format(len(userIdToOldSubreddits))

Processing ../bigData/userSubreddit/RC_2018-01
Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 6000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000
Processed 13000000
Processed 14000000
Processed 15000000
Processed 16000000
Processed 17000000
Processing ../bigData/userSubreddit/RC_2018-02
Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 13000000
Processed 14000000
Processed 15000000
Processed 16000000
Processed 17000000
Processing ../bigData/userSubreddit/RC_2018-03
Processed 2000000
Processed 3000000
Processed 4000000
Processed 5000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000
Processed 13000000
Processed 14000000
Processed 16000000
Processed 17000000
Processed 18000000
Processi

In [6]:
with open('../bigData/finalGeneration/inputGraph', 'w') as outfile:
    for userId, subreddits in userIdToOldSubreddits.iteritems():
        for subreddit, count in subreddits.iteritems():
            outfile.write("{} {} {}\n".format(userId, subreddit, count))

Outputs:
    - inputGraph

### Process June

First load the June data, with the filters.

In [9]:
infilename = "../bigData/userSubreddit/RC_2018-06"
userIdToNewSubreddits = collections.defaultdict(lambda: collections.defaultdict(int))
tools.getUserIdToSubreddits(infilename, userIdToNewSubreddits, subredditIdsToKeep, userIdsToFilter, True)
print "Number of users: {}".format(len(userIdToNewSubreddits))

Processing ../bigData/userSubreddit/RC_2018-06
Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 5000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000
Processed 13000000
Processed 14000000
Processed 15000000
Processed 16000000
Processed 17000000
Processed 18000000
Processed 19000000
Number of users: 3649104


In [11]:
# Builds actualNewSubreddits, answer key for recommendations, in JSON format
# {userId: <string>, newSubreddits: {<stringSubreddit>: <intCount>, ...}}
with open('../bigData/finalGeneration/actualNewSubreddits', 'w') as outfile:
    for userId, newSubreddits in userIdToNewSubreddits.iteritems():
        if userId not in userIdToOldSubreddits:
            continue
            
        userJson = {"userId": userId, "newSubreddits": {}}
        for subredditId, count in newSubreddits.iteritems():
            if subredditId not in userIdToOldSubreddits[userId]:
                userJson["newSubreddits"][subredditId] = count
        outfile.write(json.dumps(userJson) + "\n")

Outputs:
    - actualNewSubreddits

### Generate Dev and Test set

In [2]:
# Load inputGraph, include weights.
userIdToOldSubreddits = collections.defaultdict(lambda: collections.defaultdict(int))
tools.getUserIdToSubreddits("../bigData/finalGeneration/inputGraph", userIdToOldSubreddits, includeCounts=True)
print "Number of users: {}".format(len(userIdToOldSubreddits))

Processing ../bigData/finalGeneration/inputGraph
Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 5000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000
Processed 13000000
Processed 14000000
Processed 15000000
Processed 16000000
Processed 17000000
Processed 18000000
Processed 19000000
Processed 20000000
Processed 21000000
Processed 22000000
Processed 23000000
Processed 24000000
Processed 25000000
Processed 26000000
Processed 27000000
Processed 28000000
Processed 29000000
Processed 30000000
Processed 31000000
Processed 32000000
Processed 33000000
Processed 34000000
Processed 35000000
Processed 36000000
Processed 37000000
Processed 38000000
Processed 39000000
Processed 40000000
Processed 41000000
Processed 42000000
Processed 43000000
Processed 44000000
Processed 45000000
Processed 46000000
Processed 47000000
Processed 48000000
Processed 49000000
Processed 50000000
Processed 

In [4]:
# Load actualNewSubreddits, include weights.
actualNewSubreddits = tools.getUserIdToSubredditsByType("../bigData/finalGeneration/actualNewSubreddits", "newSubreddits")
print "Number of users: {}".format(len(actualNewSubreddits))

Processed 1000000
Processed 2000000
Number of users: 2812982


In [5]:
# Output some random subset for dev and test.
# Format is {userId: <string>, oldSubreddits: {<stringSubreddit>: <intCount>, ...}, newSubreddits: {<stringSubreddit>: <intCount>, ...}}
random.seed(7224)

candidateUserIds = []
for userId, subreddits in actualNewSubreddits.iteritems():
    if len(subreddits) >= 10 and len(subreddits) <= 100:
        candidateUserIds.append(userId)
random.shuffle(candidateUserIds)        
print "Number of candidate users: {}".format(len(candidateUserIds))

with open('../bigData/finalGeneration/devUsers', 'w') as devfile, \
     open('../bigData/finalGeneration/testUsers', 'w') as testfile:
    counter = 0
    for userId in candidateUserIds:
        userJson = {"userId": userId, 
                    "newSubreddits": actualNewSubreddits[userId],
                    "oldSubreddits": userIdToOldSubreddits[userId]}
        if counter < 100:
            devfile.write(json.dumps(userJson) + "\n")
        elif counter < 200:
            testfile.write(json.dumps(userJson) + "\n")
        else:
            break
        
        counter += 1

Number of candidate users: 114314
